In [1]:
import os
import time
import random

import cv2

import pymiere
from pymiere import wrappers
from pymiere.wrappers import get_system_sequence_presets
from pymiere.wrappers import time_from_seconds    
from pymiere.wrappers import check_active_sequence
from pymiere.wrappers import list_sequences
from pymiere.wrappers import list_video
from pymiere.wrappers import edit_clip
from pymiere.exe_utils import start_premiere

In [2]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [3]:
def buildAnimeDicPath(animeSongPath):
    animeFolderList = [ f.path for f in os.scandir(animeSongPath) if f.is_dir() ]#os.listdir(animeSongPath)
    animeDicPath = {}

    for folder in os.listdir(animeSongPath): # build dictionnary of anime folder and song path in
        animeDicPath[folder]= [ f.path for f in os.scandir(animeSongPath + '\\' + folder) if f.is_file() ]

    return animeDicPath

In [4]:
def getRandomSongPath(animeDicPath, batchNumber, songByBatch):# Create a dictionary of list of song 
    blindTestSongDic = {}
    blindTestSongDic = {'blindTest ' + str(i): [] for i in range(1,batchNumber +1)} # inizialise number of batch

    for key, value in blindTestSongDic.items():
        for songIndex in range(songByBatch):# Number of song in a batch
            
            playlist = list(animeDicPath.keys())
            animeNumber = len(playlist)
            
            playlist = playlist[random.randint(0,animeNumber)-1]#get a random anime
            songPathList = animeDicPath[playlist] #get list of song path for that playlist
            playlistSongNumber = len(songPathList) #get number of song for that anime

            if playlistSongNumber != 0: #Folder has to have a song DL at the begining
                print
                song = songPathList[random.randint(0,playlistSongNumber)-1]

                animeDicPath[playlist].remove(song)
                if len(animeDicPath[playlist]) == 0:
                    animeDicPath.pop(playlist)
                blindTestSongDic[key].append(song)
            else:
                print('No song in the playlist at the begining: ', playlist)
                animeDicPath.pop(playlist)
                pass


    return blindTestSongDic #TODO: CAN HAVE SAME SONG, TO CHECK

#### Trim block

In [5]:
def getLastClip(sequence,videoTrackIndex):
    #Video
    lastClipVideo = sequence.videoTracks[videoTrackIndex].clips[-1]

    #Audio
    lastClipAudio = sequence.audioTracks[videoTrackIndex].clips[-1]

    return lastClipVideo, lastClipAudio

def trimEnd(videoClip, audioClip, videoClipTrack3 = None, audioClipTrack3 = None, endShift = 0):
    if videoClipTrack3 and audioClipTrack3: # If a countdown clip exists, then trim w.r.t
        #Video
        videoClip.end = time_from_seconds(videoClipTrack3.end.seconds + endShift)

        #Audio
        audioClip.end = time_from_seconds(audioClipTrack3.end.seconds + endShift)
    else: # If none, then flat trim
        #Video
        videoClip.end = time_from_seconds(videoClip.end.seconds - endShift)

        #Audio
        audioClip.end = time_from_seconds(audioClip.end.seconds - endShift)

def trimStart(videoClip, audioClip, startShift):
    # fpsClip = lastClipVideo.projectItem.getFootageInterpretation().frameRate
    # shiftFrame = cutStartSecond * fpsClip
    # lenClip = lastClipVideo.end.seconds - lastClipVideo.start.seconds

    #Move clip LEFT
    # try:
    #     edit_clip(lastClipVideo, start_on_timeline=round(sequence.videoTracks[1].clips[-1].end.seconds*fpsClip), end_on_timeline=(lenClip-cutStartSecond)*fpsClip + round(sequence.videoTracks[1].clips[-1].end.seconds*fpsClip), in_point_on_clip=cutStartSecond*fpsClip, out_point_on_clip=(lenClip)*fpsClip, fps = fpsClip)
    #     edit_clip(lastClipAudio, start_on_timeline=round(sequence.videoTracks[1].clips[-1].end.seconds*fpsClip), end_on_timeline=(lenClip-cutStartSecond)*fpsClip + round(sequence.videoTracks[1].clips[-1].end.seconds*fpsClip), in_point_on_clip=cutStartSecond*fpsClip, out_point_on_clip=(lenClip)*fpsClip, fps = fpsClip)
    # except:                    
    #     edit_clip(lastClipVideo, start_on_timeline=0, end_on_timeline=(lenClip-cutStartSecond)*fpsClip + 0, in_point_on_clip=cutStartSecond*fpsClip, out_point_on_clip=(lenClip)*fpsClip, fps = fpsClip)
    #     edit_clip(lastClipAudio, start_on_timeline=0, end_on_timeline=(lenClip-cutStartSecond)*fpsClip + 0, in_point_on_clip=cutStartSecond*fpsClip, out_point_on_clip=(lenClip)*fpsClip, fps = fpsClip)
    pass

def trimClip(videoClip, audioClip, videoClipTrack3 = None, audioClipTrack3 = None, startShift = 0, endShift = 0):#TO MODIFY
    if startShift != 0:
        trimStart(videoClip, audioClip, startShift)
    if endShift != 0:
        trimEnd(videoClip, audioClip, videoClipTrack3, audioClipTrack3, endShift)

#### Scaling block

In [6]:
def getVideoRes(videoPath):#TODO: Find a way to get clip resolution through premiere pro and not windows, to rename getClipRes
    vcap = cv2.VideoCapture(videoPath) # 0=camera
 
    if vcap.isOpened(): 
        # get vcap property 
        width  = int(vcap.get(3))  # float `width`
        height = int(vcap.get(4))  # float `height`
        fps = vcap.get(5)

    return width, height, fps

def getSequenceRes(sequence):
    return sequence.frameSizeHorizontal, sequence.frameSizeVertical, 0 #TODO: find Sequence FPS

def getSeqClipRatio(sequence, videoPath):
    sequenceWidth,sequenceHeight, sequenceFPS = getSequenceRes(sequence)
    videoWidth, videoHeight, videoFPS = getVideoRes(videoPath)

    sequenceRes = sequenceWidth * sequenceHeight
    videoRes = videoWidth * videoHeight

    seqVidRatio = sequenceWidth/videoWidth*100#sequenceRes / videoRes * 100 # transform as percentage

    return seqVidRatio

def adjustScaleClip(clip, scaleValue):
    for component in clip.components:  
        if component.displayName == 'Motion':
            for property in component.properties:
                #print(property.displayName)
                if property.displayName == 'Scale':
                    property.setValue(scaleValue, True)

#### Main block

In [7]:
def createMainClip(project, sequence, blindTestList, fxDic, doCut):
    cutStartSecond, cutEndSecond = 0, 15 #TODO: if cutStartSecond !=0, video are not synchronised

    countDownVideoTrack = 2 

    for index, songPath in enumerate(blindTestList):
        #Import video on main track
        songItem = project.rootItem.findItemsMatchingMediaPath(songPath, ignoreSubclips=False)[0]
        try:
            sequence.insertClip(songItem, time_from_seconds(sequence.videoTracks[countDownVideoTrack].clips[-1].end.seconds),0, 0)
        except:
            sequence.insertClip(songItem, time_from_seconds(0),0, 0)

        lastClipVideo, lastClipAudio = getLastClip(sequence,videoTrackIndex = 0)
        
        #Adjust ratio if width are different
        ratio = getSeqClipRatio(sequence, songPath)
        adjustScaleClip(lastClipVideo, ratio)

        if ratio != 100:
            print(ratio)

        sequence.insertClip(project.rootItem.findItemsMatchingMediaPath(fxDic[1], ignoreSubclips=False)[0], time_from_seconds(sequence.videoTracks[0].clips[-1].start.seconds), countDownVideoTrack, countDownVideoTrack)
        #sequence.insertClip(project.rootItem.findItemsMatchingMediaPath(fxDic[2], ignoreSubclips=False)[0], time_from_seconds(sequence.videoTracks[countDownVideoTrack].clips[-1].end.seconds), countDownVideoTrack, countDownVideoTrack)

        if doCut: #cut clip start and end
            lastClipVideo3, lastClipAudio3 = getLastClip(sequence,videoTrackIndex = countDownVideoTrack)
            #lastClipVideo3, lastClipAudio3 = None, None
            trimClip(lastClipVideo, lastClipAudio, lastClipVideo3, lastClipAudio3, cutStartSecond, cutEndSecond)#TODO: TO MODIFY, CUT DOES NOT WORK AS INTENDED

        #Waiting video in between main clip, This need to be last on the task list
        if index < len(blindTestList)-1:#Do not create waiting countdown at the last video
            sequence.insertClip(project.rootItem.findItemsMatchingMediaPath(fxDic[0], ignoreSubclips=False)[0], time_from_seconds(sequence.videoTracks[0].clips[-1].end.seconds), countDownVideoTrack,  countDownVideoTrack) 

        sequence.audioTracks[countDownVideoTrack].setMute(1)

In [8]:
animeSongPath = r"D:\Videos\Blind Test\Anime"
animeSongList = getListOfFiles(animeSongPath)

In [9]:
doGetRandomSong = True

batchNumber = 1
songByBatch = 50

if doGetRandomSong:
    animeDicPath = buildAnimeDicPath(animeSongPath)
    blindTestSongDic = getRandomSongPath(animeDicPath, batchNumber, songByBatch)
else:
    pass

In [11]:
blindTestSongDic

{'blindTest 1': ['D:\\Videos\\Blind Test\\Anime\\Evangelion\\Neon Genesis Evangelion Opening (Zankoku na Tenshi no TEEZE) with Closed Captions.mp4',
  'D:\\Videos\\Blind Test\\Anime\\bakemonogatari\\Bakemonogatari - op 4 PL [Sengoku Nadeko].mp4',
  'D:\\Videos\\Blind Test\\Anime\\Shokugeki no Soma\\Food Wars! The Fourth Plate - Opening (HD).mp4',
  'D:\\Videos\\Blind Test\\Anime\\Saiki Kusuo no Psi nan\\The Disastrous Life of Saiki K – Opening Theme 1 – Youth Isnt So Cruel.mp4',
  'D:\\Videos\\Blind Test\\Anime\\Rave Master\\Groove Adventure Rave op.mp4',
  'D:\\Videos\\Blind Test\\Anime\\Kyoukai no Kanata\\Kyoukai No Kanata OP 60 FPS.mp4',
  'D:\\Videos\\Blind Test\\Anime\\Ken le Survivant\\Générique  Ken le Survivant.mp4',
  'D:\\Videos\\Blind Test\\Anime\\Toradora\\Toradora! - Lost My Pieces (OST).mp4',
  'D:\\Videos\\Blind Test\\Anime\\Hyouka\\Hyouka Opening 1.mp4',
  'D:\\Videos\\Blind Test\\Anime\\Dr Stone\\ＴＶアニメ 「DrSTONE」 第1クールOP＜Good Morning World!＞ノンクレジット映像.mp4',
  'D:\\Videos

In [12]:
answerTiming = r"D:\Videos\Premiere Pro\Project\BlindTest\EFX\720p\answer 720p.mp4"
countdownInterWaiting = r"D:\Videos\Premiere Pro\Project\BlindTest\EFX\1080p\cdwait2s.mp4"
countdownHid20 = r"D:\Videos\Premiere Pro\Project\BlindTest\EFX\720p\countdown20s720p.mp4"

fxDic = [countdownInterWaiting, countdownHid20, answerTiming]

In [13]:
#Create a project
project = pymiere.objects.app.project

#Create sequences and bins
#sequences = wrappers.list_sequences()
sequence_preset_path = r"C:\Program Files\Adobe\Adobe Premiere Pro 2020\Settings\SequencePresets\HDV\HDV 720p30.sqpreset"#r"C:\Program Files\Adobe\Adobe Premiere Pro 2020\Settings\SequencePresets\ARRI\1080p\ARRI 1080p 23.976fps.sqpreset"#TODO: find preset with 60 fps

#Create Bins
for sequenceNumb in range(1, batchNumber + 1):
    #Create bin
    project.rootItem.createBin('blindTest ' + str(sequenceNumb))

    #Import Medias
    project.importFiles(
        blindTestSongDic['blindTest ' + str(sequenceNumb)], 
        suppressUI=True, 
        targetBin = project.rootItem.children[sequenceNumb - 1], 
        importAsNumberedStills=False) 

project.rootItem.createBin('EFX')
pymiere.objects.app.project.rootItem.children[-1].setColorLabel(1) #set color for EFX bin

project.importFiles(fxDic, # can import a list of media  
suppressUI=True,  
targetBin = project.rootItem.children[project.rootItem.children.numItems-1],  #project.getInsertionBin()
importAsNumberedStills=False) 
#project.activeSequence = sequences[0]

#Create sequences
for sequenceNumb in range(1, batchNumber + 1):
    sequence_name = list(blindTestSongDic.keys())[sequenceNumb - 1]
    pymiere.objects.qe.project.newSequence(sequence_name, sequence_preset_path) 

create sequences

In [14]:
sequences = pymiere.objects.app.project.sequences #[s for s in pymiere.objects.app.project.sequences ] pymiere.objects.app.project.sequences

for keyIndex in range(len(blindTestSongDic.keys())):
    blindTestNumber = list(blindTestSongDic.keys())[keyIndex]
    print(blindTestNumber)
    sequence = [s for s in sequences if s.name == blindTestNumber][0] #pymiere.objects.qe.project.newSequence(sequence_name, sequence_preset_path) 

    createMainClip(project, sequence, blindTestList = blindTestSongDic[blindTestNumber], fxDic = fxDic, doCut = True)

blindTest 1
400.0
106.66666666666667
266.66666666666663
200.0
266.66666666666663
400.0
266.66666666666663
200.0


In [15]:
clipVideo = sequence.videoTracks[0].clips[-1]
clipAudio = sequence.audioTracks[0].clips[-1]

In [16]:
sequences = pymiere.objects.app.project.sequences
sequence = sequences[0]

In [17]:
#songItem = project.rootItem.findItemsMatchingMediaPath(blindTestSongDic['blindTest 1'][0], ignoreSubclips=False)[0]

#### Audio Compressor

In [18]:
def applyTubeCompressor(clip, threshold = .8, ratio = .07, gain = 0.534, autoGain = 0.534, Attack = 1, release = 10):
    for component in clip.components:
        if component.displayName == 'Tube-modeled Compressor':
            for property in component.properties:
                if 'Threshold' in property.displayName:#TODO: Not working, set as -60db
                    property.setValue(threshold, True) #0.9 <=> -6db // .783 <=> -13db // .8 <=> -12db
                if 'Ratio' in property.displayName:#TODO: Not working, set as -60db
                    property.setValue(ratio, True)# .007 <=> ratio 3:1 // .105 <=> 4:1
                if 'Attack' in property.displayName:#TODO: Not working, set as -60db
                    property.setValue(Attack, True)# .007 <=> ratio 3:1
                if 'Release' in property.displayName:#TODO: Not working, set as -60db
                    property.setValue(release, True)# .007 <=> ratio 3:1
                if 'Auto Makeup Gain' in property.displayName:#TODO: Not working, set as -60db
                    property.setValue(autoGain, True)# .007 <=> ratio 3:1
                # if 'Gain' in property.displayName:#TODO: Not working, set as -60db
                #     property.setValue(gain, True)# 0.534 <=> 2db // 0.583 <=> 5db

In [19]:
for i in qe.getAudioTransitionList():
    print(i)

NameError: name 'qe' is not defined

#### add compressor to all song in all sequence in audio tracks 1

In [ ]:
pymiere.objects.app.enableQE()

False

In [ ]:
qe_project.getAudioEffectByName("Tube-modeled Compressor")

TypeError: 'PymiereGenericObject' object is not iterable

In [19]:
qe_project = pymiere.objects.qe.project 

In [20]:
def applyAudioTrackTransition(audioTrackQE, strTransitionName, strTransitionTime):#apply audio transition to whole audio track
    for clipAudioIndex in range(audioTrackQE.numItems):
        if clipAudioIndex %2 != 0: # apply at the end of the audio clip
            clipAudioQE = audioTrackQE.getItemAt(clipAudioIndex)
            clipAudioQE.addTransition(qe_project.getAudioTransitionByName(strTransitionName), True,strTransitionTime)
    print('Audio transition done')

def applyVideoTrackTransition(videoTrackQE, strTransitionName, strTransitionTime):
    for clipVideoIndex in range(videoTrackQE.numItems):#Video track
        if clipVideoIndex %2 != 0: # apply at the end of the video clip
            clipVideoQE = videoTrackQE.getItemAt(clipVideoIndex)
            clipVideoQE.addTransition(qe_project.getVideoTransitionByName(strTransitionName), True,strTransitionTime)
    print('Video transition done')

In [21]:
for sequenceIndex in range(qe_project.numSequences):
    sequenceQE = qe_project.getSequenceAt(sequenceIndex)
    audioTrackQE = sequenceQE.getAudioTrackAt(0)  
    videoTrackQE = sequenceQE.getVideoTrackAt(0)  

    applyAudioTrackTransition(audioTrackQE, "Constant Power", '25')
    applyVideoTrackTransition(videoTrackQE, "Cross Dissolve", '10')

Audio transition done
Video transition done


In [22]:
# clip = pymiere.objects.app.project.activeSequence.audioTracks[0].clips[0]  

sequences = pymiere.objects.app.project.sequences

for sequence in sequences:
    print('-------------------' + sequence.name + '-------------------')
    for audioClip in sequence.audioTracks[0].clips:
        applyTubeCompressor(audioClip, threshold = .8, ratio = .105, gain = 0.583, autoGain = 0.0, Attack = 0.02, release = 0.02)

-------------------blindTest 1-------------------


In [21]:
clip = pymiere.objects.app.project.activeSequence.audioTracks[0].clips[0]  
# find our effect in components  
for component in clip.components: 
    print(component.displayName)

    for property in component.properties:
        print(property.displayName)

Volume
Bypass
Level
Channel Volume
Bypass
Left 
Right 


In [30]:
pymiere.objects.app.project.activeSequence.createMotionGraphicsTemplate("My Text Template")

AttributeError: 'Sequence' object has no attribute 'createMotionGraphicsTemplate'

In [33]:
for i in pymiere.objects.app.project.activeSequence.videoTracks[0].clips[0].components:
    print(i.displayName)

Opacity
Motion


In [ ]:
for i in qe_project.getAudioTransitionList():
    print(i)

Constant Power
Constant Gain
Exponential Fade


In [ ]:
for i in qe_project.getVideoTransitionList():
    print(i)

Additive Dissolve
Band Slide
Band Wipe
Barn Doors
Center Merge
Center Peel
Center Split
Channel Map
CheckerBoard
Checker Wipe
Clock Wipe
Cross Dissolve
Cross Stretch
Cross Zoom
Cube Spin
Curtain
Dip to Black
Dip to White
Displace
Dither Dissolve
Doors
Film Dissolve
Flip Over
Fold Up
Gradient Wipe
Inset
Iris Cross
Iris Diamond
Iris Points
Iris Round
Iris Shapes
Iris Box
Iris Star
Luminance Map
Multi-Spin
Non-Additive Dissolve
Page Peel
Page Turn
Paint Splatter
Peel Back
Pinwheel
Push
Radial Wipe
Random Blocks
Random Invert
Random Wipe
Roll Away
Slash Slide
Slide
Sliding Bands
Sliding Boxes
Spin
Spin Away
Spiral Boxes
Split
Stretch
Stretch In
Stretch Over
Swap
Swing In
Swing Out
Swirl
Texturize
Three-D
Tumble Away
Venetian Blinds
Wedge Wipe
Wipe
Zig-Zag Blocks
Zoom
Zoom Boxes
Zoom Trails
Cross Dissolve
Dip to Black
Dip to White
Morph Cut
VR Chroma Leaks
VR Gradient Wipe
VR Iris Wipe
VR Light Leaks
VR Light Rays
VR Mobius Zoom
VR Random Blocks
VR Spherical Blur


constant Power : interpose 2 clips pour une meilleur transition
constant gain:
exponential fade:

In [ ]:
clip.mediaType

'Audio'

In [ ]:
# clip = pymiere.objects.app.project.activeSequence.videoTracks[0].clips[0]  
# # find our effect in components  
# for component in clip.components:  
#     print(component.displayName)
#     if component.displayName == "Simple Text":  
#         break  

# for property, property_name in zip(component.properties, ["?", "Position", "Justification", "Size", "Opacity", "Content"]):  
#     if property_name == "Position":  
#         property.setValue([0, 0], True)  
#     if property_name == "Content":  
#         property.setValue("NARUTO", True)

In [ ]:
pymiere.objects.app.anywhere.listProductions()

In [ ]:
clip = pymiere.objects.app.project.activeSequence.videoTracks[0].clips[0]  

effect = clip.components
for component in effect:
    print('----' + component.displayName + '----')
    for property in component.properties:
        print(property.displayName)

----Opacity----
Opacity
Blend Mode
Blend Mode
----Motion----
Position
Scale
Scale Width
 
Rotation
Anchor Point
Anti-flicker Filter


In [ ]:
# qe_project = pymiere.objects.qe.project 

# # get first clip of first video track  
# track = qe_project.getActiveSequence().getVideoTrackAt(0)  
# for x in range(track.numItems):  
#     clip = track.getItemAt(x)  
#     if clip.type != "Empty":  
#         break  
# # add Twirl video effect on clip
# clip.addVideoEffect(qe_project.getVideoEffectByName("Simple Text"))